In [1]:
import load
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import tree
#import makegal

In [2]:
from general import defaults
#from galaxymodule import quick_mock
dfl = defaults.Default()
dfl.dir_repo

'/home/hoseung/Work/pyclusterevol/repo/'

In [3]:
from galaxymodule import make_gal

In [4]:
cd ~/Work/data/Horizon-AGN/

/media/hoseung/btrfs/Horizon-AGN


In [71]:
cd ~/Work/data/NH/

/home/hoseung/Work/data/NH


In [5]:
nout = 782#312#782 # 312
s = load.sim.Sim(nout=nout)
gcat = tree.halomodule.Halo(nout=nout, is_gal=True)

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.
[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =./


In [6]:
#gg = load.rd_GM.Gal(nout, catalog=gcat.data[21].copy(), info=s.info)
gg = load.rd_GM.Gal(nout, catalog=gcat.data[104].copy(), info=s.info)
gg.debug=False
make_gal.mk_gal(gg)
print(np.log10(gg.meta.mstar))
#gg1.cal_norm_vec()
#gg1.meta.nvec

info <load.info.Info object at 0x7fa1b36d56d8>
File Not Found: ./halo/HAL_00782/hal_dms_0000105
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000105
No CELL data loaded
catalog center -44.2949333190918 51.372005462646484 23.004234313964844
star x [-44.26342773 -44.33200455 -44.30377579 ..., -44.29323578 -44.29428864
 -44.29762268]
star x [ 31.50558472 -37.07122803  -8.84246826 ...,   1.69754028   0.64468384
  -2.68936157]
gal.debug False
11.8439622296


In [7]:
import time

In [54]:
arr = np.arange(4)
mat = np.random.rand(4,3)


In [93]:
mat

array([[ 0.83086452,  0.23718903,  0.32862803],
       [ 0.2366768 ,  0.33746605,  0.75426078],
       [ 0.1154754 ,  0.3073384 ,  0.19447749],
       [ 0.74724667,  0.62018871,  0.83527797]])

In [105]:
np.tensordot(mat.T, arr, axes=1)

array([ 2.70936759,  2.81270899,  3.64904966])

In [92]:
np.multiply(mat.T, arr)

array([[ 0.        ,  0.2366768 ,  0.23095079,  2.24174   ],
       [ 0.        ,  0.33746605,  0.6146768 ,  1.86056613],
       [ 0.        ,  0.75426078,  0.38895498,  2.5058339 ]])

In [118]:
def tonumpyarray(mp_arr):
    return np.frombuffer(mp_arr.get_obj())

from multiprocessing import Process, Value, Array
import numpy as np
class Simplemock():
    def __init__(self, repo="/home/hoseung/Work/pyclusterevol/repo/sed/",
                 filter_system="SDSS",
                 sed_model="bc03",
                 load=True, 
                 parallel=True):
        self.filter_system = filter_system
        self.repo = repo
        self.sed_model = sed_model
        self.parallel = parallel
        if load:
            self.load_all()
            
    def load_all(self):
        self.load_filters()
        self.load_SED_wavelength()
        return self.load_SED_all()

    def load_filters(self):
        if self.filter_system == "SDSS":
            filter_lambda, filter_u, filter_g, filter_r, filter_i, filter_z = \
                np.genfromtxt(self.repo + "filter_sdss.dat",
                                skip_header=1, unpack=True)

            self.filters = {"lambda":filter_lambda,
                            "u":filter_u,
                            "g":filter_g,
                            "r":filter_r,
                            "i":filter_i,
                            "z":filter_z}

    def load_SED_wavelength(self):
        if self.sed_model == "bc03":
            self.sed_wavelength = np.genfromtxt(self.repo + "lambda.dat")

    def load_SED_all(self):
        """
        Full SEDs are just a few tens of MB.
        """
        if self.sed_model == "bc03":
            self.metal_points = np.array([0.0004, 0.001, 0.004, 0.01, 0.02, 0.04])
            # age points in tables.
            self.age_points = np.genfromtxt(self.repo+"ages_yybc.dat") # in Gry unit
            
            
            if self.parallel:
                SED_shared=Array('d', np.zeros(6*221*1221))
                self.SEDs = tonumpyarray(SED_shared).reshape(6,221,1221)
                print(self.SEDs.shape)
            else:
                self.SEDs = np.zeros((6, 221, 1221))
                
            
            for i, metal in enumerate(self.metal_points):
                #self.SEDs[i,:,:] = np.genfromtxt(self.repo +
                self.SEDs[i,:,:] = np.genfromtxt(self.repo +
                                            "bc03_yy_{:.4f}".format(metal)).reshape(221, 1221)

            if self.parallel:
                return SED_shared
            #else:
            #    return SEDs
        else:
            print("Sorry, Only bc03 is implemented.")

def get_flux(extinction = False,
             metal_lower_cut = False,
             filter_name='r'):
    #import gc
    #import os
    ### star data ########################################################
    global star
    global SEDs
    
    #print("PID ",os.getpid(), "reference counter", len(gc.get_referrers(star)))
    
    starmetal = star["metal"] # Is the original array modified?
    if metal_lower_cut:
        # No star with metallicity lower than the lowest table.
        # Modifying array will result in meory copy!!
        starmetal[starmetal < min(self.metal_points)] = min(self.metal_points) * 1.0001

    locate_metal = np.digitize(starmetal, self.metal_points)-1 # GOOD
    relevant_metals = self.metal_points[:max(locate_metal)+2]
    nmetals = len(relevant_metals)

    # Star Age
    t_univ = 13.7
    starage = t_univ - star["time"]

    locate_age = np.digitize(starage, self.age_points)-1 # GOOD
    relevant_ages = self.age_points[:max(locate_age)+2]
    nages = len(relevant_ages)

    ### Filter optimization. #################################################

    # Pick one
    this_filter = self.filters[filter_name]

    # band range
    i_filter_pos = this_filter > 0

    this_filter = this_filter[i_filter_pos]
    filter_lambda_this_band = self.filters["lambda"][i_filter_pos]

    lambda_min_this_band = min(filter_lambda_this_band)
    lambda_max_this_band = max(filter_lambda_this_band)

    i_lambda_min = np.argmax(self.sed_wavelength > lambda_min_this_band) -1
    #print(i_lambda_min, wavelength[i_lambda_min], lambda_min_this_band)
    i_lambda_max = np.argmax(self.sed_wavelength > lambda_max_this_band)
    #print(i_lambda_max, wavelength[i_lambda_max], lambda_max_this_band)

    # Only a small part of SED is needed.
    wavelength = self.sed_wavelength[i_lambda_min:i_lambda_max+1]
    n_wavelength = i_lambda_max - i_lambda_min + 1

    ##### Caclulate band flux #################
    # Load only necessary data
    # Load all once, keep under the class and copy part of it when needed heere.
    seds = np.zeros((nmetals, nages, n_wavelength)) # metal age lambda
    nages_org =221
    nmetal_org=6
    n_wavs_org=1221
    if self.sed_model == "bc03":
        for i, metal in enumerate(relevant_metals):
            for j, age in enumerate(relevant_ages):
                seds[i,j,:] = SEDs[ i * nages_org*n_wavs_org
                                  + j * n_wavs_org + i_lambda_min:
                                    i * nages_org*n_wavs_org
                                  + j * n_wavs_org + i_lambda_max+1]

    # All are array-wise calculations.
    # interpolation weight
    dmda = np.sqrt((relevant_metals[locate_metal+1] - relevant_metals[locate_metal]) * (relevant_ages[locate_age+1] - relevant_ages[locate_age]))
    dl_m = (starmetal - relevant_metals[locate_metal] )# / \
           #(relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
    dr_m = (relevant_metals[locate_metal+1] - starmetal)# / \
           #(relevant_metals[locate_metal+1] - relevant_metals[locate_metal])
    dl_a = (starage - relevant_ages[locate_age] )#   / \
           #(relevant_ages[locate_age+1] - relevant_ages[locate_age])
    dr_a = (relevant_ages[locate_age+1] - starage )# / \
           #(relevant_ages[locate_age+1] - relevant_ages[locate_age])

    # 2D linear interpolation
    # weight * SED.
    print(dmda.shape)
    if True:
        Flux = np.multiply(1/dmda,
            np.multiply( (dr_m * dr_a), seds[locate_metal, locate_age,:].T) +\
            np.multiply( (dl_m * dr_a), seds[locate_metal + 1, locate_age,:].T) +\
            np.multiply( (dr_m * dl_a), seds[locate_metal, locate_age + 1, :].T) +\
            np.multiply( (dl_m * dl_a), seds[locate_metal + 1, locate_age + 1,:].T))
        #print(Flux.shape)
    else:
        Flux = (np.multiply(seds[locate_metal, locate_age,:], (dr_m * dr_a)) +\
               np.multiply(seds[locate_metal + 1, locate_age,:], (dl_m * dr_a)) +\
               np.multiply(seds[locate_metal, locate_age + 1, :], (dr_m * dl_a)) +\
               np.multiply(seds[locate_metal + 1, locate_age + 1,:], (dl_m * dl_a)))/dmda
        #print(Flux.shape)

    # Convolve filter
    # Wavelengths at which filter function are defined are different from the SED wavelength points.
    # Interpolate filter function on SED points.
    filter_in_sed_wavelengths = np.interp(wavelength, filter_lambda_this_band, this_filter)
    Flux = np.multiply(filter_in_sed_wavelengths, Flux)
    
    if not extinction:
        return np.sum(Flux, axis=1)
    else:
        print("Extinction - Not yet implemented")
        return


In [153]:
SED_shared=Array('d', np.zeros((6*221*1221)))#, lock=False)
SEDs = tonumpyarray(SED_shared).reshape(6,221,1221)
print(SEDs.shape)

(6, 221, 1221)


In [20]:
MockSED = Simplemock(repo=dfl.dir_repo+'sed/', load=False)
SEDs = MockSED.load_all()
global self
global SEDs
#SEDs = MockSED.SEDs.copy()
MockSED.SEDs=0
self = MockSED#.SEDs
global star
star = gg.star
#gg.star["metal"][gg.star["metal"] >= 0.04] = 0.04 - 1e-9
## Improve Boundary cases. !

(6, 221, 1221)


Even everything to be shared are global, 
having a subprocess referencing to the original memory modifies the original memory.
That's because Python needs to count references to a memory so that it can clean up the memory when no other objects are referencing the memory. 
That means, just having one more subprocess means another copy of memory. 

A great article on this problem. 
https://llvllatrix.wordpress.com/2016/02/19/python-vs-copy-on-write/


#### The core of our solution is to tell the operating system that it should be using shared memory that isn’t reference counted. 

In [119]:
import multiprocessing as mp
import gc

mp.set_start_method('fork', force=True)

Ps = []
# args must be a tuple. (something,) note the trailing comma.
#Ps.append(mp.Process(target = get_flux, kwargs={"filter_name":'u'}))
#Ps.append(mp.Process(target = get_flux, kwargs={"filter_name":'g'}))
#Ps.append(mp.Process(target = get_flux, kwargs={"filter_name":'r'}))
#Ps.append(mp.Process(target = get_flux, kwargs={"filter_name":'i'}))
#Ps.append(mp.Process(target = get_flux, kwargs={"filter_name":'z'}))

t0 = time.time()

for filtername in ['u','g','r','i','z']:
    p=mp.Process(target = get_flux, kwargs={"filter_name":filtername})
    p.start()
    #print(len(gc.get_referrers(MockSED)))
    Ps.append(p)
    
for p in Ps:
    p.join()

print(time.time()-t0)

/home/hoseung/Work/hspy/lib/python3.5/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in sqrt


(200516,)


/home/hoseung/Work/hspy/lib/python3.5/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in sqrt


(200516,)


/home/hoseung/Work/hspy/lib/python3.5/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in sqrt


...

(200516,)


/home/hoseung/Work/hspy/lib/python3.5/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in sqrt


(200516,)


Process Process-141:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-118-25afadb03634>", line 167, in get_flux
    Flux = np.multiply(filter_in_sed_wavelengths, Flux)
ValueError: operands could not be broadcast together with shapes (58,) (58,200516) 
Process Process-143:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-118-25afadb03634>", line 167, in get_flux
    Flux = np.multiply(filter_in_sed_wavelengths, Flux)
ValueError: operands could not be broadcast together with shapes (92,) (92,200516) 
Process Process-142:
Traceback (most recent call las

1.8866689205169678


In [30]:
t0 = time.time()
Flux_u = get_flux(filter_name='u')
Flux_g = get_flux(filter_name='g')
Flux_r = get_flux(filter_name='r')
Flux_i = get_flux(filter_name='i')
Flux_z = get_flux(filter_name='z')
print(time.time()-t0)

1.9056005477905273


In [48]:
def launch_jobs(shared_data, num_jobs=5, num_worker=4):
    global shared_data
    data_array = shared_data

    pool = multiprocessing.Pool(num_worker)
    return pool.map(job_handler, range(num_jobs))
            


SyntaxError: name 'shared_data' is parameter and global (<ipython-input-48-79065e72f4f6>, line 2)

In [132]:
# Can rotate galaxy. 
gg1.reorient(pops=["star"])

In [ ]:
Fluxs = [Flux_u, Flux_g, Flux_r, Flux_i, Flux_z]
draw(Fluxs, gg1.star["x"], gg1.star["y"], suffix="face")
draw(Fluxs, gg1.star["x"], gg1.star["z"], suffix="edge")


Unit + observation condition.